In [2]:
import tensorflow as tf
import numpy as np
import rocket_tools as rt
from keras import backend 
from keras.backend import learning_phase
import cv2
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

Using TensorFlow backend.


In [3]:
image_size = 128
n_images = 3
#steps = 500

tf.reset_default_graph()

x_input = tf.placeholder(tf.float32,
                         [None, image_size, image_size, n_images],
                         name='input')


model = rt.model.U_NET(input_=(image_size, image_size, n_images))
y_pred = model(x_input)

sess = tf.InteractiveSession()
saver = tf.train.Saver()

MODEL_PATH = './../saved_models/u_net_image_size=128_snr_all_w_more_dot'
saver.restore(sess, MODEL_PATH)

INFO:tensorflow:Restoring parameters from ./../saved_models/u_net_image_size=128_snr_all_w_more_dot


In [3]:
snr_test = np.linspace(1, 10.0, 100)
print('start')
for snr in snr_test:
    results={}
    print(snr)
    predictions = []
    true_masks = []

    image_generator = rt.image_gen.ImageGenerator(
        image_size=(image_size, image_size), snr_mean=snr, snr_std=0.0)
    batch_generator = rt.batch_generator.BatchGenerator(image_generator, batch_size=32, n_images=n_images)
    
    
    for step in range(steps):
        x_array, y_true_array = next(batch_generator)
        y_pred_array = sess.run(y_pred, feed_dict={
            x_input: x_array,
            learning_phase(): 1
        })
        predictions.append(y_pred_array)
        true_masks.append(y_true_array)
        
    snr_r = round(snr, 1)
    results['snr_{}'.format(snr_r)] = {
        'true': true_masks,
        'pred': predictions
    }
    np.savez_compressed('exp_res/resulst_snr_{}.npz'.format(snr_r), results=results)
print('finish')
#np.savez_compressed('resulst_snr_{}_100.npz'.format(snr), results=results)

start
1.0
1.09090909091
1.18181818182
1.27272727273
1.36363636364
1.45454545455
1.54545454545
1.63636363636
1.72727272727
1.81818181818
1.90909090909
2.0
2.09090909091
2.18181818182
2.27272727273
2.36363636364
2.45454545455
2.54545454545
2.63636363636
2.72727272727
2.81818181818
2.90909090909
3.0
3.09090909091
3.18181818182
3.27272727273
3.36363636364
3.45454545455
3.54545454545
3.63636363636
3.72727272727
3.81818181818
3.90909090909
4.0
4.09090909091
4.18181818182
4.27272727273
4.36363636364
4.45454545455
4.54545454545
4.63636363636
4.72727272727
4.81818181818
4.90909090909
5.0
5.09090909091
5.18181818182
5.27272727273
5.36363636364
5.45454545455
5.54545454545
5.63636363636
5.72727272727
5.81818181818
5.90909090909
6.0
6.09090909091
6.18181818182
6.27272727273
6.36363636364
6.45454545455
6.54545454545
6.63636363636
6.72727272727
6.81818181818
6.90909090909
7.0
7.09090909091
7.18181818182
7.27272727273
7.36363636364
7.45454545455
7.54545454545
7.63636363636
7.72727272727
7.81818181818


IOError: Failed to write to /home/kimsvimpel/Students/sosnovik/u-rocket/exp_res/resulst_snr_8.0.npztXc1Gz-numpy.npy: [Errno 28] No space left on device

In [13]:
results={}
n_images = 3
steps = 100
pred = []
true_msk = []
image_generator = rt.image_gen.ImageGenerator(image_size=(image_size, image_size), snr_mean=6, snr_std=2)
batch_generator = rt.batch_generator.BatchGenerator(image_generator, batch_size=1, n_images=n_images)
for step in range(steps):
    x_array, y_true_array = next(batch_generator)
    y_pred_array = sess.run(y_pred, feed_dict={
        x_input: x_array,
        learning_phase(): 1 
        })
    
    pred.append(y_pred_array)
    true_msk.append(y_true_array)
        
    results['snr_all'] = {
        'true': true_msk,
        'pred': pred
    }
np.savez_compressed('exp_res/empty_res_new.npz', results=results)